# Flask SQLAlchemy - Instrukcje DML w relacjach

W bazach relacyjnych tabele mogą być powiązane ze sobą relacjami. W zależności od natury tego powiązania wyróżniamy kilka typów relacji. W Django ORM możemy wyróżnić trzy podstawowe typy relacji: jeden-do-jednego, jeden-do-wielu i wiele-do-wielu. Każda z tych relacji posiada w Django ORM swój odpowiednik w postaci pola modelu. I tak:
* **OneToOneField** to pole odpowiadające relacji jeden do jednego
* **ForeignKey** to pole odpowiadające relacji jeden do wielu
* **ManyToManyField** to pole odpowiadające relacji wiele do wielu

Omówmy je po kolei.

In [1]:
# Setup
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy.orm import DeclarativeBase

class Base(DeclarativeBase):
  pass

db = SQLAlchemy(model_class=Base)

# create the app
app = Flask(__name__)

# configure the SQLite database, relative to the app instance folder
app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///database.sqlite3"

# initialize the app with the extension
db.init_app(app)

app.app_context().push()

## OneToOneField

Relacja jeden do jednego występuje w przypadku kiedy rekord jednej tabeli może być powiązany z jednym i tylko jednym wpisem drugiej tabeli. Przykładem takiej relacji może być tabela stolica oraz tabela państwo. Warszawa jest stolicą tylko jednego państwa - Polski, Polska ma przypisaną tylko jedną stolicę - Warszawę.
"Mówimy Warszawa myślimy Polska, mówimy Polska myślimy Warszawa". Relacja jest symetryczna dlatego nie ma znaczenia, w której z tabelek (stolica, czy państwo) umieścimy kolumnę dla tej relacji.

**Definicje modeli**

W modelach mamy dwie klasy: Country i Capitol. Relacje OneToOneField umieściliśmy po stronie modelu Country. Atrybut przechowujący relację nazwaliśmy capitol.

<code>class Country(models.Model):
    name = models.CharField(max_length=64)
    capitol = models.OneToOneField('Capitol', on_delete=models.CASCADE)
</code>

<code>class Capitol(models.Model):
    name = models.CharField(max_length=64)
</code>

### C z CRUD

In [2]:
from sqlalchemy import Integer, String
from sqlalchemy.orm import Mapped, mapped_column
from sqlalchemy import UniqueConstraint 


class Country(db.Model):
    id: Mapped[int] = db.Column(db.Integer, primary_key=True)
    name: Mapped[str] = db.Column(db.String(64), nullable=False)
    capital_id: Mapped[int] = db.Column(db.Integer, db.ForeignKey('capital.id'), unique=True)
    # capital_id: Mapped[int] = db.Column(db.Integer, db.ForeignKey('capital.id'), uselist=False)  # same effect
    
    def __repr__(self):
        return f'<Country "{self.name}">'


class Capital(db.Model):
    id: Mapped[int] = db.Column(db.Integer, primary_key=True)
    name: Mapped[str] = db.Column(db.String(64), nullable=False)

    # `db.relationship` does not impact the structure of the database table; 
    # rather, it is solely intended for convenient referencing.
    country: db.Mapped['Country'] = db.relationship("Country", backref="capital")
    
    def __repr__(self):
        return f'<Capitol "{self.name}">'


db.create_all()

In [3]:
# 1. Tworzymy wpis w tabeli Capital
warsaw = Capital(name="Warsaw")
db.session.add(warsaw)
db.session.commit()

In [4]:
# 2. Tworzymy wpis w tabeli Country
poland = Country(name="Poland")
db.session.add(poland)
db.session.commit()

In [5]:
# Stworzyliśmy wpis do tabeli Country bez wartości w polu capital_id.

# Spróbujmy z wartością
paris = Capital(name="Paris")
france = Country(name="France", capital_id=2)
db.session.add_all([
    paris,
    france
])
db.session.commit()

In [6]:
paris.country  # to byłoby niemożliwe, gdyby nie wprowadzenie db.relationship

<Country "France">

In [7]:
france.capital  # i to też

<Capitol "Paris">

In [8]:
germany = Country(name="Germany", capital_id=2)
db.session.add(germany)
db.session.commit()  # Integrity Error

IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: country.capital_id
[SQL: INSERT INTO country (name, capital_id) VALUES (?, ?)]
[parameters: ('Germany', 2)]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

In [9]:
# db.session.rollback()

In [10]:
# Możemy też powiązanie stworzyć w ten sposób:
rome = Capital(name="Rome")
italy = Country(name="Italy", capital=rome)  # to byłoby niemożliwe bez db.relationship
db.session.add_all([
    rome,
    italy
])
db.session.commit()

In [11]:
# lub tak
madrit = Capital()
madrit.name = "Madrit"

spain = Country()
spain.name = "Spain"
spain.capital = madrit

db.session.add_all([
    madrit,
    spain
])
db.session.commit()

## ForeignKey

Relacja jeden do wielu jest najczęściej wykorzystywanym typem relacji. Występuje wtedy, kiedy wpis z jednej tabeli (tzw. tabeli rodzica) może być powiązany z wieloma wpisami z drugiej (tzw. tabeli dziecka), ale wpis z drugiej tabeli (tabeli dziecka) nie może być powiązany z wielom wpisami z pierwszej (tabeli rodzica). Innymi słowy rodzic może mieć wiele dzieci, ale dziecko może mieć tylko jednego rodzica. Przykładem takiej relacji może być tabela "miasto" oraz tabela "państwo". Gdańsk należy do Polski. Kraków należy do Polski. Mówiąc Gdańsk myślimy Polska (to strona relacja "jeden"). Podobnie mówiąc Kraków myślimy Polska. Ale już mówiąc Polska myślimy Gdańsk, Kraków, Wrocław, ... (to strona relacji "wiele"). 

Najprościej jest wyobrazić sobie relacje jeden-do-wielu jako strukturę hierarchiczną, czyli drzewo. Na górze mamy rodzica, a pod nim wiele dzieci. Tutaj Polska jest rodzicem, a Gdańsk, Kraków, Wrocław, ... dziećmi. To po której stronie umieścimy pole do przechowywania relacji zależy wyłącznie od nas. Zazwyczaj znacznie łatwiej myśli się o takiej relacji, kiedy pole umieści się po stronie dziecka (ponieważ dziecko ma tylko jednego rodzica). Czyli umieszczamy pole ForeignKey w modelu "miasto" i wpisy Gdańsk, Kraków, Wrocław posiadają referencje do wpisu Polska z modelu "państwo". 

Innym przykładem takiej relacji może być język programowania i framework. Jezyk programowania to np. Python. Framework to np. Django, Flask, Bottle. Rodzicem jest tu język programowania, dziećmi poszczególne frameworki (mówimy Python myślimy Django, Flask, Bottle... ale mówimy Django myślimy Python). Czyli pole z relacją najlepiej umieścić po stronie modelu Framework.

**Definicje modeli**

W modelach mamy dwie klasy: Language i Framework. Relacje ForignKey umieściliśmy po stronie modelu Framework (dziecko). Atrybut przechowujący relację nazwaliśmy language.

<code>class Language(models.Model):
    name = models.CharField(max_length=64)
</code>
<code>
    def __str__(self):
        return f"{self.name}"
</code>

<code>class Framework(models.Model):
    name = models.CharField(max_length=64)
    language = models.ForeignKey('Language', on_delete=models.CASCADE)
</code>
<code>
    def __str__(self):
        return f"{self.name} ({self.language})"
</code>

In [12]:
from sqlalchemy import Integer, String
from sqlalchemy.orm import Mapped, mapped_column


class Language(db.Model):
    id: Mapped[int] = db.Column(db.Integer, primary_key=True)
    name: Mapped[str] = db.Column(db.String(64), nullable=False)
    
    # annotation change (list)
    frameworks: db.Mapped[list['Framework']] = db.relationship("Framework", backref="language")
    
    def __repr__(self):
        return f'<Language "{self.name}">'


class Framework(db.Model):
    id: Mapped[int] = db.Column(db.Integer, primary_key=True)
    name: Mapped[str] = db.Column(db.String(64), nullable=False)
    language_id: Mapped[int] = db.Column(db.Integer, db.ForeignKey('language.id'))

    def __repr__(self):
        return f'<Framework "{self.name}">'
    

db.create_all()

### C z CRUD

Operacje "C" niczym się nie różnią od tych dla pola OneToOneField. W ramach utrwalenia

In [13]:
# Metad I (instancja modelu)

python = Language(name="python")
django = Framework(name="django", language=python)
flask = Framework(name="flask", language=python)
fastapi = Framework(name="fastapi", language=python)

db.session.add_all([
    python,
    django,
    flask,
    fastapi
])
db.session.commit()

In [15]:
# Metad II (id instancji modelu)

java = Language(name="java")
spring = Framework(name="spring", language_id=java.id)

db.session.add_all([
    java,
    spring
])
db.session.commit()

C:\Users\User\AppData\Local\Temp\ipykernel_35876\288772681.py:10: SAWarning: New instance <Language at 0x2e0d8db1310> with identity key (<class '__main__.Language'>, (1,), None) conflicts with persistent instance <Language at 0x2e0d93580d0>
  db.session.commit()


IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: language.id
[SQL: INSERT INTO language (id, name) VALUES (?, ?)]
[parameters: (1, 'java')]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

### R z CRUD

Na początek wyświetlmy wszystkie frameworki.

In [17]:
Framework.query.all()

[<Framework "django">,
 <Framework "flask">,
 <Framework "fastapi">,
 <Framework "spring">]

In [18]:
# A frameworki tylko dla konkretnego języka?
# Klasyczny filtr po polu language.

# 1. Pobieramy obiekt, po którym będziemy wyszukiwać.
python = Language.query.filter_by(name='python').first()

# 2. Wyszukujemy po pobranym obiekcie.
frameworks = Framework.query.filter_by(language=python).all()
print(frameworks)

[<Framework "django">, <Framework "flask">, <Framework "fastapi">]


In [19]:
# wartością atrybutu reprezentującego relację jest obiekt
django = Framework.query.filter_by(name='django').first()
print(django.language)
print(type(django.language))

<Language "python">
<class '__main__.Language'>


In [20]:
# Realizacja relacji odwrotnej w SQLAlchemy
python = Language.query.filter_by(name='python').first()
print(python.frameworks)

[<Framework "django">, <Framework "flask">, <Framework "fastapi">]


In [21]:
python = Language.query.filter_by(name='python').first()
print(python.frameworks)

frameworks = Framework.query.filter(Framework.language==python, Framework.name.like("fl%")).all()
print(frameworks)

[<Framework "django">, <Framework "flask">, <Framework "fastapi">]
[<Framework "flask">]


## ManyToMany

Ostatnia z omawianych relacji to relacja wiele-do-wielu. Dotyczy sytuacji kiedy wpisy z jednej tabeli mogą być powiązane z wielom wpisami z drugiej oraz wpisy z drugiej tabeli mogą być powiązane z wieloma wpisami z tabeli pierwszej. Przykładem takiej relacji może być tabela Aktor oraz tabela Film. Mówimy Al Pacino myślimy Scareface, Gorączka, Ojciec Chrzestny... Mówimy Ojciec Chrzestny myślimy Al Pacino, Robert DeNiro, Marlon Brando ... Relację możemy umieścić w dowolnej z powiązanych tabeli.
Jest to najbardziej złożony typ relacji, ponieważ zgodnie z zasadami normalizacji realizacja relacji wiele-do-wielu wymaga wprowadzenia tabeli pośredniej.

**Definicje modeli**

W modelach mamy dwie klasy: Actor i Movie. Relacje ManyToMany umieszczamy po stronie Movie (ale równie dobrze moglibyśmy umieścić po stronie Actor). Atrybut przechowujący relację nazwaliśmy actors (zwróć uwagę na to, że tym razem w nazwie pola użyliśmy liczby mnogiej).

<code>class Actor(models.Model):
    name = models.CharField(max_length=64)
</code>
<code>
    def __str__(self):
        return f"{self.name}"
</code>

<code>class Movie(models.Model):
    title = models.CharField(max_length=128)
    actors = models.ManyToManyField('Actor')
</code>
<code>
    def __str__(self):
        return f"{self.title}"
</code>


In [22]:
# It is strongly recommended to use a table instead of a model for many-to-many relationship (so called helper table).
actor_movie = db.Table('actor_movie',
    db.Column('actor_id', db.Integer, db.ForeignKey('actor.id'), primary_key=True),
    db.Column('movie_id', db.Integer, db.ForeignKey('movie.id'), primary_key=True)
)

class Actor(db.Model):    
    id: Mapped[int] = db.Column(db.Integer, primary_key=True)
    name: Mapped[str] = db.Column(db.String(128), nullable=False)
    movies: db.Mapped[list['Movie']] = db.relationship("Movie", secondary=actor_movie, backref="actors")  # plurals    
    
    def __repr__(self):
        return f'<Actor "{self.name}">'

class Movie(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name: Mapped[str] = db.Column(db.String(128), nullable=False)
    
    def __repr__(self):
        return f'<Movie "{self.name}">'


db.create_all()

In [23]:
movie_1 = Movie(name='The Godfather')
movie_2 = Movie(name='The Heat')
movie_3 = Movie(name='The Irishman')
movie_4 = Movie(name='Taxi Driver')
movie_5 = Movie(name='Matrix')

actor_1 = Actor(name='Al Pacino')
actor_2 = Actor(name='Robert De Niro')
actor_3 = Actor(name='Keanu Reeves')

db.session.add_all([
    movie_1,
    movie_2,
    movie_3,
    movie_4,
    movie_5,
    actor_1,
    actor_2,
    actor_3
])
db.session.commit()

Patrzymy do bazy i widzimy wpisy w tabeli movie i wpisy w tabeli actor, ale w tabeli movie_actors mamy 0 wpisów. W jaki sposób powiązać teraz wpisy z tabeli movie z wpisami z tabeli actor? 

In [24]:
# Sposób I
from sqlalchemy import insert

stmt = (
    insert(actor_movie).
    values(actor_id=actor_1.id, movie_id=movie_1.id)
)
db.session.execute(stmt)
db.session.commit()

In [25]:
print(actor_1.movies)

[<Movie "The Godfather">]


In [26]:
print(movie_1.actors)

[<Actor "Al Pacino">]


In [27]:
# Sposób II
actor_2.movies.append(movie_1)
db.session.commit()

In [28]:
print(actor_2.movies)

[<Movie "The Godfather">]


In [29]:
print(movie_1.actors)

[<Actor "Al Pacino">, <Actor "Robert De Niro">]


In [30]:
# Sposób III
movie_5.actors.append(actor_3)
db.session.commit()

In [31]:
print(movie_5.actors)

[<Actor "Keanu Reeves">]


Podsumowując

In [32]:
# Wszystkie filmy w których zagrał Al Pacino
actor_1.movies

[<Movie "The Godfather">]

In [33]:
# Wszyscy aktorzy, którzy zagrali w 'The Godfather'
movie_1.actors

[<Actor "Al Pacino">, <Actor "Robert De Niro">]

### Usuwanie powiązań

In [34]:
print(movie_1.actors)

[<Actor "Al Pacino">, <Actor "Robert De Niro">]


In [35]:
movie_1.actors.remove(actor_2)
db.session.commit()

In [36]:
print(movie_1.actors)

[<Actor "Al Pacino">]


## Zlączenia (join)

In [37]:
result = db.session.query(Framework, Language).join(Language).all()
print(result)

[(<Framework "django">, <Language "python">), (<Framework "flask">, <Language "python">), (<Framework "fastapi">, <Language "python">)]


In [38]:
result = db.session.query(Actor, Movie, actor_movie).select_from(actor_movie).join(Actor).join(Movie).all()
print(result)

[(<Actor "Al Pacino">, <Movie "The Godfather">, 1, 1), (<Actor "Keanu Reeves">, <Movie "Matrix">, 3, 5)]


In [39]:
result = db.session.query(Actor, Movie, actor_movie).select_from(Actor).join(actor_movie).join(Movie).all()
print(result)

[(<Actor "Al Pacino">, <Movie "The Godfather">, 1, 1), (<Actor "Keanu Reeves">, <Movie "Matrix">, 3, 5)]


In [ ]:
result = db.session.query(Actor, Movie, actor_movie).select_from(Actor).join(actor_movie).join(Movie).filter(Actor.id==1).all()
print(result)